```
From: https://github.com/ksatola
Version: 0.0.1

TODOs
1. https://machinelearningmastery.com/calculate-feature-importance-with-python/
2. https://towardsdatascience.com/explaining-feature-importance-by-example-of-a-random-forest-d9166011959e
3. 
```

# Feature Importance
Bagged decision trees like Random Forest and Extra Trees can be used to estimate the importance of features. In the example below we construct a ExtraTreesClassifier classifier for the Pima Indians onset of diabetes dataset.

In [2]:
# Connect with underlying Python code
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(0, '../src')

In [3]:
from datasets import (
    get_dataset,
)

In [4]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

In [6]:
# Feature Importance with Extra Trees Classifier
from sklearn.ensemble import ExtraTreesClassifier

# target = 'class'
df = get_dataset('pima-indians-diabetes')
df.head()
array = df.values
X = array[:,0:8]
y = array[:,8]

# Feature extraction
model = ExtraTreesClassifier(n_estimators=10)
model.fit(X, y)
print(model.feature_importances_)

[0.11430284 0.21227502 0.10805356 0.08138834 0.07882261 0.1401769
 0.11318618 0.15179454]


You can see that we are given an importance score for each attribute where the larger score the more important the attribute. The scores suggest at the importance of plas, age and mass.